In [14]:
import numpy as np
import pandas as pd
import plfit
import powerlaw
from numpy.random import rand,seed

In [5]:
X=plfit.plexp_inv(rand(5000),100,2.5)

C:\Users\Krist\Anaconda3\lib\site-packages\plfit\plfit.py:805: RuntimeWarning: invalid value encountered in power
  Ppl = lambda X: 1+C*(xmin/(1-alpha)*(X/xmin)**(1-alpha))


In [6]:
myplfit=plfit.plfit(X,usefortran=False)

C:\Users\Krist\Anaconda3\lib\site-packages\plfit\plfit.py:118: RuntimeWarning: divide by zero encountered in double_scalars
  a = 1+float(n) / sum(log(x/xmin))


PYTHON plfit executed in 2.017143 seconds
xmin: 102.42 n(>xmin): 639 alpha: 2.49887 +/- 0.0592943   Log-Likelihood: -3764.7   ks: 0.0148494 p(ks): 0.747052


In [15]:
pvalues, results, stats = getpowerlaw(X, times = 1, extend=True)

C:\Users\Krist\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: divide by zero encountered in double_scalars


The number of subdatasets, where the power-law could not be fitted within the chosen tolerance level, are 0 and the total number of subdatasets are 1


In [16]:
results

,0
Alpha,2.49887
Sigma,0.0592943
KS statistic,0.0148494
Xmin,102.42
#ofdatapoints,639


In [7]:
# This implementation uses the method of minimizing the distance between the empirical CDF and the CDF of the best-fit 
# model to determine the appropiate value of xmin. 

def getpowerlaw(data,tollevel = 0.1,times =2500,extend = False, validate = True): 
    
    if isinstance(data,pd.DataFrame)==False:
        data = pd.DataFrame(data)
    
    notfitted = 0
    
    holder = pd.DataFrame(columns = data.columns,index = ['Alpha','Sigma','KS statistic','Xmin','#ofdatapoints'])
    p_values = []
    simstats = []
    for column in data:
        
        # The printing can be useful to understand where in the fitting process the algorithm are.
        
        #print('It is dataset number %i that is being processed' % int(column))
        
        columndata = data[column]
        columndata = columndata.sort_values()
        
        if extend:
            
            ksvalues,sigmas,alphas,xmins,datapoints,datatype = modelfitter(columndata, tollevel, extend = True)
             
        else:
            
            ksvalues,datatype = modelfitter(columndata, tollevel, extend = False)
        
        # Locating the specifications that caused the minimum distance between the empirical and theoretical distribution.
        if len(ksvalues)==0:
            
            # Public-service announcement
            
            #print('The power-law can not be fitted within the tolerance level chosen')
            notfitted += 1
            continue
            
        bestks = np.argmin(ksvalues)
        
        # If no tail a present, the data can't be taught to follow a power-law distribution
        if xmins[bestks]==data[column].min():
            continue

        holder[column]['Alpha'] = alphas[bestks]
        holder[column]['Sigma'] = sigmas[bestks]
        holder[column]['KS statistic'] = ksvalues[bestks]
        holder[column]['Xmin'] = xmins[bestks]
        holder[column]['#ofdatapoints'] = datapoints[bestks]

        # The user has the opportunity to get the data validated, i.e. that it actually comes from a power law distribution.
        # We know go on and use the best-fit specifications to validate if our data indeed does follow a power-law.
        
        if validate == True:

            if extend:
                nonpowerlawdata = columndata[columndata<xmins[bestks]]
                p_val, simstat = validatepowerlaw(alphas[bestks], xmins[bestks], ksvalues[bestks], len(columndata),
                                                   datapoints[bestks],columndata[columndata<xmins[bestks]],
                                                   times, datatype, tollevel, extend = extend)
            else:
                nonpowerlawdata = columndata[columndata<xmins[bestks]]
                p_val = validatepowerlaw(alphas[bestks], xmins[bestks], ksvalues[bestks], len(columndata),datapoints[bestks],
                                     columndata[columndata<xmins[bestks]], times, datatype, tollevel, extend = extend)
            p_values.append(p_val)
            simstats.append(simstat)
    print('The number of subdatasets, where the power-law could not be fitted within the chosen tolerance level, are %i and the total number of subdatasets are %i' % (notfitted,len(data.columns)))
    
    if notfitted == len(data.columns):
        return 0,0,0
    
    if extend == True and validate == True:

        return p_values,holder,simstats

    elif extend == False and validate == True:

        return p_values,holder
        
    elif extend == False and validate == False:

        return holder

In [8]:

def validatepowerlaw(alpha,xmin,bestks,lenoforiginaldata,lenoftaildata,data,times = 2500,
                     datatype = 'Discrete',tollevel=0.1,extend = False):
    p_value = 0
    
    a, x = alpha,xmin
    N = lenoforiginaldata
    N_xmin = lenoftaildata
    prob1 = N_xmin/N
    prob2 = 1-prob1
    
    # Drawing the synthetic data that is needed to validate the hypothesis about our data following a power-law distribution.
    
    samples = samplesyntheticdata(data,N,prob1,prob2,times,a,x,datatype)
    
    # If the user wants information on each of the simulation, he shall get it - it follows down the code.
    
    if extend:
        simulatedstats = pd.DataFrame(columns = samples.columns,
                                      index = ['Alpha','Sigma','KS statistic','Xmin','#ofdatapoints'])
    
    # For each sample, we fit over power law and calculate the KS statistic.
    
    for sample in samples:
        
        cursample = samples[sample].sort_values()
        
        if extend:
            # The part that goes out of the area: modelfitter(cursample, tollevel,extend = True)
            ksvalues_sample,sigmas_sample,alphas_sample,xmins_sample,datapoints_sample, datatype = modelfitter(cursample, tollevel,
                                                                                                               extend = True)
            
        else:
            
            ksvalues_sample,datatype = modelfitter(cursample, extend = False)
       
        # If no test statistic could be calculated, a new sample is drawn until a test statistic is calculated.
        if len(ksvalues_sample) == 0:
            holdme = sample
            j = 0
            
            while(len(ksvalues_sample)==0):
                
                # The printing can be useful to understand the dynamics of the fitting.
                
                #print('It is the %i iteration in the request of a sample which is useful' % (j+1))
                
                sample = samplesyntheticdata(data,N,prob1,prob2,1,a,x,datatype)
                
                cursample = sample[sample.columns[0]].sort_values()
                
                if extend:
                    # The part that goes out of the area: modelfitter(cursample, tollevel,extend = True)
                    ksvalues_sample,sigmas_sample,alphas_sample,xmins_sample,datapoints_sample, datatype = modelfitter(cursample, tollevel,
                                                                                                                       extend = True)
            
                else:
            
                    ksvalues_sample, datatype = modelfitter(cursample, extend = False)
                        
                j += 1
            
            sample = holdme
            
            # The printing can be useful to understand the dynamics of the fitting.
            
            #print('It took %i iterations to get a sample which was useful' % (j+1))
                    
        bestks_sample = np.argmin(ksvalues_sample)
        
        if extend:
            
            simulatedstats[sample]['Alpha'] = alphas_sample[bestks_sample]
            simulatedstats[sample]['Sigma'] = sigmas_sample[bestks_sample]
            simulatedstats[sample]['KS statistic'] = ksvalues_sample[bestks_sample]
            simulatedstats[sample]['Xmin'] = xmins_sample[bestks_sample]
            simulatedstats[sample]['#ofdatapoints'] = datapoints_sample[bestks_sample]
        
        if ksvalues_sample[bestks_sample]>bestks:
            
            p_value += 1.0
    
    if extend:
        
        return p_value/float(times),simulatedstats
    
    else:
        
        return p_value/float(times)

In [9]:
def samplesyntheticdata(data,lenoforiginaldata,prob1,prob2,times,alpha,xmin,datatype):
    
    # Firstly, we draw all the necessary draws from the left-out observations
    
    sample_1 = pd.DataFrame(np.random.choice(data,(round(prob2*lenoforiginaldata),times)))
    
    #Initiallizing the dataframe to contain the total amount of draws
    
    samples_2 = pd.DataFrame()
    
    # Drawing our synthetic data, to be used for getting the p-values for the hypothesis of if our data comes from 
    # a power-law distribution.
    if times > 1:
        
        for t in np.arange(times):

            samples_2[t] = powerlaw.Power_Law(xmin=xmin, parameters=[alpha]).generate_random(round(prob1*lenoforiginaldata))
    else:
        
        samples_2[0] = powerlaw.Power_Law(xmin=xmin, parameters=[alpha]).generate_random(round(prob1*lenoforiginaldata))
    
    samples_3=pd.concat([samples_2,sample_1],ignore_index=True)
    
    # If the data is discrete data, we round of the data, consistent with what is proposed in [Clauset et al., 2009]
    
    if datatype == 'Discrete':
        samples_3 = samples_3.round()
    
    return samples_3

In [10]:
def modelfitter(data, tollevel = 0.1,extend = False):

    ksvalues = []
        
    if extend:

        sigmas = []
        xmins = []
        alphas = []
        datapoints = []

    for point in np.unique(data):
        
        # If the xmin candicate is zero, the calculations breaks down.
        if point == 0:
            continue

        currentdata = data[data >=point]
        n = len(currentdata)

        # Chosing the appropiate estimator for alpha

        if (data.min()/int(data.min()) == 1 and data.median()/int(data.median())==1):

            datatype = 'Discrete'

            # Approximate alpha expression based on the assumption that integers are approximated as
            # continous reals rounded to the nearest integer. (Formula 3.7)

            alpha = (1+n*(sum(np.log(currentdata/(point-0.5)))**(-1)))

        else:

            datatype = 'Continuous'
            alpha = (1+n*(sum(np.log(currentdata/point))**(-1)))

        # The estimator for Alpha is asymptotic normal distributed and consistent.
        # Therefore it is possible to evaluate the sigma of the estimator based on the normal p-values.

        # Making a conservative adjustment if the sample size is particular small

        if len(currentdata)<50:

            sigma = (alpha - 1)/np.sqrt(n) + (1/n)

        else:

            sigma = (alpha - 1)/np.sqrt(n)

        # We disregard estimates of alpha that results in a higher standard error that 0.1.
        # The reason here of is fact that we drop observations every time we examin a new xmin, and
        # we achknowledge that we are working few observations, so a higher tolerance level is accpeted..
        
        if sigma < tollevel:

            cdfres = np.arange(len(currentdata))/len(currentdata)

            # Evaluation of the Complimentary Cumulative distribution function in calculation of the KS statistic

            bestfitpowerCDF = 1-(currentdata.sort_values()/point)**(-alpha+1)

            ksvalues.append(max(abs(bestfitpowerCDF-cdfres)))

            if extend:

                sigmas.append(sigma)
                alphas.append(alpha)
                xmins.append(point)
                datapoints.append(n)

        else:

            break
            
    if extend:

        return ksvalues,sigmas,alphas,xmins,datapoints,datatype

    else:

        return ksvalues,datatype

In [ ]:
# Generate power-law distributed artificial data, to test the above inplementation

alpha, xmin = 2.5, 100

# These probabilities are for chosen arbitrary

# Uncomment this if you want to sample some of the distribution from a distribution of your own choice.
# Furthermore, uncomment the two outcommented lines below, one in the for-loop and one right after.

#tail = pd.DataFrame()
#neck = pd.DataFrame(C*np.random.exponential(lamb,(round(prob2*N),times)))
#prob1 = 0.8
#prob2 = 1-prob1
#lamb = 0.5
#C = np.exp(lamb*xmin)

N = 2500
times = 5000

artificialdata = pd.DataFrame()

start = time.clock()

for gen in np.arange(times):
        
        #tail[gen] = powerlaw.Power_Law(xmin=xmin, parameters=[alpha]).generate_random(round(prob1*N))
        artificialdata[gen] = plfit.plexp_inv(rand(N),xmin,alpha)

# If you need continuous data, just remove/outcomment the rounding.

#artificialdata = round(pd.concat([tail,neck],ignore_index=True))
artificialdata = round(artificialdata)        

end = time.clock()

print('It took %.3f to generate %i synthetic dataset of %i observations' % ((end-start),times,N))